In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize']=(12,8)

In [2]:
def replace_string(data, column):
    k_columns = data[data[column].str.contains("K")]
    k_columns = k_columns.loc[k_columns[column]!='K']
    k_columns[column] = k_columns[column].str.strip('K').astype('float') * 1000
    m_columns = data[data[column].str.contains("M")]
    m_columns = m_columns.loc[m_columns[column]!='M']
    m_columns[column] = m_columns[column].str.strip('M').astype('float') * 1e6
    b_columns = data[data[column].str.contains("B")]
    b_columns = b_columns.loc[b_columns[column]!='B']
    b_columns[column] = b_columns[column].str.strip('B').astype('float') * 1e9
    columns = pd.concat([k_columns, m_columns, b_columns], axis=0)
    
    columns = columns.sort_index()
    
    return columns

In [3]:
%%time
storm_events = []
#init_path = '/Users/allanbellahsene/Desktop/THESIS/DATA/NCFEI/StormEvents_' #Mac_OS path
init_path = 'C:\\Users\\abellahsene\\Desktop\\THESIS\\DATA\\NCFEI\\StormEvents_' #Windows path
for years in list(np.arange(1962,2020)):
    path = init_path + str(years) + '.csv'
    storm_data = pd.read_csv(path, usecols=['BEGIN_DAY', 'MONTH_NAME', 'YEAR', 'STATE', 'DAMAGE_PROPERTY', 'DAMAGE_CROPS', 'EVENT_TYPE', 'EPISODE_ID', 'EVENT_ID', 'DEATHS_DIRECT', 'DEATHS_INDIRECT'])
    storm_data.index = pd.to_datetime(storm_data['BEGIN_DAY'].astype('str')+ '-' + storm_data['MONTH_NAME'] + '-' + storm_data.YEAR.astype('str'))
    storm_data = storm_data[['STATE', 'DAMAGE_PROPERTY', 'DAMAGE_CROPS', 'DEATHS_DIRECT', 'DEATHS_INDIRECT', 'EVENT_TYPE', 'EVENT_ID', 'EPISODE_ID']] #keep only relevant columns
    storm_data.dropna(inplace=True) #drop missing values
    storm_data.sort_index(inplace=True) #sort data by date
    storm_events.append(storm_data)
    #print(years)
STORMS = pd.concat([storm_events[0], storm_events[1]], axis=0)
for i in range(2, len(storm_events)):
    STORMS = pd.concat([STORMS, storm_events[i]], axis=0)
print(STORMS.shape)
df = STORMS
df['DAMAGE_PROPERTY'] = df['DAMAGE_PROPERTY'].astype('str')
df['DAMAGE_CROPS'] = df['DAMAGE_CROPS'].astype('str')
STORMS = df
STORMS = STORMS.loc[((STORMS.DAMAGE_PROPERTY!='0') & (STORMS.DAMAGE_CROPS =='0')) | ((STORMS.DAMAGE_PROPERTY=='0') & (STORMS.DAMAGE_CROPS !='0')) | ((STORMS.DAMAGE_PROPERTY!='0') & (STORMS.DAMAGE_CROPS !='0'))]
STORMS = STORMS.loc[STORMS.DAMAGE_PROPERTY!='K'] #delete rows that only contain "K"
STORMS = STORMS.loc[STORMS.DAMAGE_CROPS!='K']
STORMS = replace_string(STORMS, 'DAMAGE_PROPERTY')
STORMS = replace_string(STORMS, 'DAMAGE_CROPS')
STORMS['TOTAL_DAMAGES'] = STORMS['DAMAGE_PROPERTY'] + STORMS['DAMAGE_CROPS']
STORMS['Date'] = STORMS.index
STORMS = STORMS[['Date','STATE', 'DAMAGE_PROPERTY', 'DAMAGE_CROPS', 'TOTAL_DAMAGES', 'DEATHS_DIRECT', 'DEATHS_INDIRECT', 'EVENT_TYPE', 'EVENT_ID', 'EPISODE_ID']]
df = STORMS.groupby(['STATE', 'Date']).agg({'TOTAL_DAMAGES':sum})
STORMS = STORMS.loc[STORMS['TOTAL_DAMAGES']!=0]
States = list(STORMS.STATE.unique())

(708372, 8)
Wall time: 17.8 s


In [4]:
STORMS.head()

,Date,STATE,DAMAGE_PROPERTY,DAMAGE_CROPS,TOTAL_DAMAGES,DEATHS_DIRECT,DEATHS_INDIRECT,EVENT_TYPE,EVENT_ID,EPISODE_ID
1972-06-18,1972-06-18,FLORIDA,190000.0,5000.0,195000.0,0,0,Tornado,990000001,990000001.0
1972-06-18,1972-06-18,FLORIDA,10000.0,5000.0,15000.0,1,0,Tornado,990000002,990000001.0
1996-01-01,1996-01-01,MISSOURI,10000.0,0.0,10000.0,0,0,Winter Storm,5546429,2031319.0
1996-01-02,1996-01-02,KANSAS,8000.0,0.0,8000.0,1,0,Winter Storm,5536209,2404253.0
1996-01-02,1996-01-02,FLORIDA,1500.0,0.0,1500.0,0,0,Thunderstorm Wind,5538451,2404206.0


In [5]:
df =pd.DataFrame(STORMS.groupby(['EVENT_TYPE']).agg({'DAMAGE_CROPS':sum}))
df.sort_index()

,DAMAGE_CROPS
EVENT_TYPE,
Avalanche,0.000000e+00
Blizzard,7.060000e+06
Coastal Flood,7.500000e+05
Cold/Wind Chill,7.715000e+08
Debris Flow,2.000910e+07
Dense Fog,0.000000e+00
Dense Smoke,0.000000e+00
Drought,1.539101e+10
Dust Devil,0.000000e+00


In [6]:
nat_disasters = list(STORMS.EVENT_TYPE.unique())

In [7]:
v_cons = {'Tornado': 'High',
           'Winter Storm': 'High',
           'Thunderstorm Wind': 'High',
           'Waterspout': 'Medium',
           'High Wind': 'High',
           'Flash Flood': 'Medium',
           'Flood': 'High',
           'Cold/Wind Chill': 'Medium',
           'Ice Storm': 'High',
           'Hail': 'High',
           'Lightning': 'High',
           'Drought': 'Medium',
           'Hurricane (Typhoon)': 'High',
           'Storm Surge/Tide': 'High',
           'Wildfire': 'High',
           'Heavy Rain': 'High',
           'Heavy Snow': 'High',
           'Blizzard': 'High',
           'Frost/Freeze': 'High',
           'Winter Weather': 'Medium',
           'Dense Fog': 'Medium',
           'Strong Wind': 'High',
           'Tropical Storm': 'High',
           'Dust Storm': 'Medium',
           'Extreme Cold/Wind Chill': 'Medium',
           'Debris Flow': 'High',
           'Coastal Flood': 'High',
           'Heat': 'Medium',
           'Marine Thunderstorm Wind': 'Low',
           'Landslide': 'High',
           'Lake-Effect Snow': 'Medium',
           'High Surf': 'Medium',
           'Lakeshore Flood': 'Low',
           'Tsunami': 'High',
           'Seiche': 'Low',
           'Marine Strong Wind': 'Low',
           'Avalanche': 'Medium',
           'Hurricane': 'High',
           'Dense Smoke': 'Low',
           'Freezing Fog': 'Medium',
           'Dust Devil': 'Medium',
           'Funnel Cloud': 'Low',
           'Marine High Wind': 'Medium',
           'Marine Hail': 'Low',
           'Tropical Depression': 'Medium',
           'Rip Current': 'Low',
           'Sleet': 'Medium',
           'Marine Hurricane/Typhoon': 'Low',
           'Marine Lightning': 'Medium',
           'Marine Dense Fog': 'Low',
           'Sneakerwave': 'Low'}
v_agr = {'Tornado': 'High',
           'Winter Storm': 'High',
           'Thunderstorm Wind': 'High',
           'Waterspout': 'Low',
           'High Wind': 'High',
           'Flash Flood': 'High',
           'Flood': 'High',
           'Cold/Wind Chill': 'High',
           'Ice Storm': 'High',
           'Hail': 'High',
           'Lightning': 'Medium',
           'Drought': 'High',
           'Hurricane (Typhoon)': 'High',
           'Storm Surge/Tide': 'Medium',
           'Wildfire': 'High',
           'Heavy Rain': 'High',
           'Heavy Snow': 'High',
           'Blizzard': 'Medium',
           'Frost/Freeze': 'Medium',
           'Winter Weather': 'Low',
           'Dense Fog': 'Low',
           'Strong Wind': 'High',
           'Tropical Storm': 'High',
           'Dust Storm': 'Medium',
           'Extreme Cold/Wind Chill': 'Medium',
           'Debris Flow': 'Medium',
           'Coastal Flood': 'Medium',
           'Heat': 'High',
           'Marine Thunderstorm Wind': 'Medium',
           'Landslide': 'Medium',
           'Lake-Effect Snow': 'Medium',
           'High Surf': 'Medium',
           'Lakeshore Flood': 'Low',
           'Tsunami': 'Medium',
           'Seiche': 'Low',
           'Marine Strong Wind': 'Low',
           'Avalanche': 'Low',
           'Hurricane': 'High',
           'Dense Smoke': 'Low',
           'Freezing Fog': 'Low',
           'Dust Devil': 'Low',
           'Funnel Cloud': 'Low',
           'Marine High Wind': 'Medium',
           'Marine Hail': 'Low',
           'Tropical Depression': 'Low',
           'Rip Current': 'Low',
           'Sleet': 'Low',
           'Hurricane (Typhoon)': 'High',
           'Marine Hurrican/Typhoon': 'Low',
           'Marine Lightning': 'Low',
           'Marine Dense Fog': 'Low',
           'Sneakerwave': 'Low'}


v_energy = {'Tornado': 'High',
           'Winter Storm': 'High',
           'Thunderstorm Wind': 'High',
           'Waterspout': 'High',
           'High Wind': 'High',
           'Flash Flood': 'High',
           'Flood': 'High',
           'Cold/Wind Chill': 'Medium',
           'Ice Storm': 'High',
           'Hail': 'Medium',
           'Lightning': 'High',
           'Drought': 'Low',
           'Hurricane (Typhoon)': 'High',
           'Storm Surge/Tide': 'High',
           'Wildfire': 'Low',
           'Heavy Rain': 'Medium',
           'Heavy Snow': 'Medium',
           'Blizzard': 'High',
           'Frost/Freeze': 'Medium',
           'Winter Weather': 'Low',
           'Dense Fog': 'Low',
           'Strong Wind': 'Medium',
           'Tropical Storm': 'Medium',
           'Dust Storm': 'Low',
           'Extreme Cold/Wind Chill': 'Low',
           'Debris Flow': 'Medium',
           'Coastal Flood': 'Medium',
           'Heat': 'Low',
           'Marine Thunderstorm Wind': 'Medium',
           'Landslide': 'Low',
           'Lake-Effect Snow': 'Low',
           'High Surf': 'Low',
           'Lakeshore Flood': 'Medium',
           'Tsunami': 'High',
           'Seiche': 'Low',
           'Marine Strong Wind': 'Medium',
           'Avalanche': 'High',
           'Hurricane': 'High',
           'Dense Smoke': 'Low',
           'Freezing Fog': 'Low',
           'Dust Devil': 'Low',
           'Funnel Cloud': 'Low',
           'Marine High Wind': 'Medium',
           'Marine Hail': 'Low',
           'Tropical Depression': 'Medium',
           'Rip Current': 'Low',
           'Sleet': 'Low',
           'Marine Hurricane/Typhoon': 'Medium',
           'Marine Lightning': 'Low',
           'Marine Dense Fog': 'Low',
           'Sneakerwave': 'Low'}


In [8]:
STORMS['Vulnerability Energy'] = STORMS['EVENT_TYPE'].map(v_energy)

In [9]:
STORMS['Vulnerability Agriculture'] = STORMS['EVENT_TYPE'].map(v_agr)
STORMS['Vulnerability Construction'] = STORMS['EVENT_TYPE'].map(v_cons)

In [10]:
STORMS

,Date,STATE,DAMAGE_PROPERTY,DAMAGE_CROPS,TOTAL_DAMAGES,DEATHS_DIRECT,DEATHS_INDIRECT,EVENT_TYPE,EVENT_ID,EPISODE_ID,Vulnerability Energy,Vulnerability Agriculture,Vulnerability Construction
1972-06-18,1972-06-18,FLORIDA,190000.0,5000.0,195000.0,0,0,Tornado,990000001,990000001.0,High,High,High
1972-06-18,1972-06-18,FLORIDA,10000.0,5000.0,15000.0,1,0,Tornado,990000002,990000001.0,High,High,High
1996-01-01,1996-01-01,MISSOURI,10000.0,0.0,10000.0,0,0,Winter Storm,5546429,2031319.0,High,High,High
1996-01-02,1996-01-02,KANSAS,8000.0,0.0,8000.0,1,0,Winter Storm,5536209,2404253.0,High,High,High
1996-01-02,1996-01-02,FLORIDA,1500.0,0.0,1500.0,0,0,Thunderstorm Wind,5538451,2404206.0,High,High,High
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-12-30,2019-12-30,WISCONSIN,384000.0,0.0,384000.0,0,0,Flood,863582,143837.0,High,High,High
2019-12-30,2019-12-30,OHIO,25000.0,0.0,25000.0,0,0,High Wind,870106,144846.0,High,High,High
2019-12-30,2019-12-30,NEW YORK,50000.0,0.0,50000.0,0,0,Strong Wind,865535,144202.0,Medium,High,High
2019-12-30,2019-12-30,OHIO,100000.0,0.0,100000.0,0,0,High Wind,870093,144846.0,High,High,High


In [17]:
np.array(STORMS.DAMAGE_PROPERTY).T

array([190000.,  10000.,  10000., ...,  50000., 100000.,  10000.])

In [22]:
def climate_score(climate_data, year, state, sector):
    df = climate_data.copy()
    df = df.loc[(df.index>=year+'-01-01') & (df.index<=year+'-12-31') & (df.STATE==state)]
    df['Vulnerability'] = df['Vulnerability '+sector].map({'High': 1, 'Medium': 0.5, 'Low': 0})
    
    score = np.dot(np.array(df.TOTAL_DAMAGES).T, np.array(df.Vulnerability))
    
    return score

In [32]:
def climate_ranking(climate_data, year, sector):
    score = []
    for states in list(climate_data.STATE.unique()):
        score.extend((states, climate_score(climate_data, year, states, sector)))
    score = np.array(score).reshape((int(len(score)/2), 2))
    ranking = pd.DataFrame(score, columns=['State', 'Climate Score Risk'])
    ranking['Climate Score Risk'] = ranking['Climate Score Risk'].astype('float')
    ranking.sort_values(by='Climate Score Risk', inplace=True, ascending=False)
    ranking.reset_index(inplace=True)
    ranking.drop(columns=['index'], inplace=True)
    
    return ranking

In [33]:
climate_ranking(climate_data=STORMS, year='2017', sector='Energy')

,State,Climate Score Risk
0,TEXAS,4.620803e+10
1,PUERTO RICO,1.902670e+10
2,FLORIDA,2.996919e+09
3,MICHIGAN,7.413652e+08
4,SOUTH CAROLINA,3.804905e+08
...,...,...
59,LAKE ONTARIO,0.000000e+00
60,LAKE ST CLAIR,0.000000e+00
61,LAKE ERIE,0.000000e+00
62,DISTRICT OF COLUMBIA,0.000000e+00


In [34]:
#Financial fundamental data

#fundamentals = pd.read_csv('/Users/allanbellahsene/Desktop/THESIS/DATA/WHARTON_DATABASE/Funda_data.csv')
path = "C:\\Users\\abellahsene\\Desktop\\THESIS\\DATA\\WHARTON_DATABASE\\Funda_data.csv"
fundamentals = pd.read_csv(path)
fundamentals[['Date']] = fundamentals[['datadate']].applymap(str).applymap(lambda s: "{}/{}/{}".format(s[4:6],s[6:], s[0:4]))
fundamentals.Date = pd.to_datetime(fundamentals.Date)
fin_data = fundamentals
sector = '11'
data = fin_data.loc[fin_data.naics.astype('str').str.startswith(sector)]
data = data[['gvkey', 'fqtr', 'tic', 'conm', 'atq', 'niq', 'loc', 'state', 'naics', 'Date']]
data.index = data.Date
data = data.loc[data['loc'] == 'USA'] #keep only companies with headquarters in the US
us_state_abbrev = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'American Samoa': 'AS',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'District of Columbia': 'DC',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Guam': 'GU',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Northern Mariana Islands':'MP',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Puerto Rico': 'PR',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virgin Islands': 'VI',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY'
}
abbrev_us_state = dict(map(reversed, us_state_abbrev.items()))
data['State'] = data['state'].map(abbrev_us_state)
data.State = data.State.str.upper()
data['ROA'] = data['niq'] / data['atq']
data = data.loc[abs(data.ROA)<2]
data = data[['gvkey', 'tic', 'conm', 'ROA','State']]

In [35]:
data

,gvkey,tic,conm,ROA,State
Date,,,,,
1981-11-30,1266,ALCO,ALICO INC,0.055831,FLORIDA
1982-08-31,1266,ALCO,ALICO INC,0.017322,FLORIDA
1982-11-30,1266,ALCO,ALICO INC,0.009589,FLORIDA
1983-02-28,1266,ALCO,ALICO INC,0.062897,FLORIDA
1983-05-31,1266,ALCO,ALICO INC,0.025911,FLORIDA
...,...,...,...,...,...
2012-05-31,187769,BOPO,BIOPOWER OPERATIONS CORP,-0.695985,FLORIDA
2012-08-31,187769,BOPO,BIOPOWER OPERATIONS CORP,-0.847887,FLORIDA
2014-02-28,187769,BOPO,BIOPOWER OPERATIONS CORP,-1.705882,FLORIDA


In [ ]:
def financial_ranking(financial_data, year, sector):
    
    df = financial_data.loc[(financial_data.index>=year+'-01-01') & (financial_data.index<=year+'-01-01')]
    